In [45]:
!conda list



# packages in environment at C:\ProgramData\Anaconda3:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0                    py38_0  
alabaster                 0.7.12             pyhd3eb1b0_0  
alembic                   1.8.1                    pypi_0    pypi
anaconda                  2021.05                  py38_0  
anaconda-client           1.7.2                    py38_0  
anaconda-navigator        2.2.0            py38haa95532_0  
anaconda-project          0.9.1              pyhd3eb1b0_1  
anyio                     2.2.0            py38haa95532_2  
appdirs                   1.4.4                      py_0  
argh                      0.26.2                   py38_0  
argon2-cffi               20.1.0           py38h2bbff1b_1  
asn1crypto                1.4.0                      py_0  
astroid                   2.5              py38haa95532_1  
astropy                   4.2.1            py38h2bbff1b_1  
async_generator           1.10

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(143): Could not remove or rename C:\ProgramData\Anaconda3\conda-meta\pip-21.0.1-py38haa95532_0.json.  Please remove this file manually (you may need to reboot to free file handles)


win_inet_pton             1.1.0            py38haa95532_0  
win_unicode_console       0.5                      py38_0  
wincertstore              0.2                      py38_0  
winpty                    0.4.3                         4  
wrapt                     1.12.1           py38he774522_1  
xlrd                      2.0.1              pyhd3eb1b0_0  
xlsxwriter                1.3.8              pyhd3eb1b0_0  
xlwings                   0.23.0           py38haa95532_0  
xlwt                      1.3.0                    py38_0  
xmltodict                 0.12.0                     py_0  
xz                        5.2.5                h62dcd97_0  
yaml                      0.2.5                he774522_0  
yapf                      0.31.0             pyhd3eb1b0_0  
zeromq                    4.3.3                ha925a31_3  
zfp                       0.5.5                hd77b12b_6  
zict                      2.0.0              pyhd3eb1b0_0  
zipp                      3.4.1         

In [35]:
import os
import logging
import yaml
import json
import requests
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
import re
import numpy as np
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import SpectralClustering
from sklearn.metrics import f1_score, silhouette_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.tracking import MlflowClient

In [44]:
!dir 

 Том в устройстве E имеет метку BigBoy
 Серийный номер тома: 0A39-EB06

 Содержимое папки e:\education\auto-pipeline-airflow-mlflow\notebooks

07.08.2022  23:43    <DIR>          .
07.08.2022  23:43    <DIR>          ..
07.08.2022  18:16    <DIR>          .ipynb_checkpoints
07.08.2022  23:43            12 148 20220807_trying_petproject4MFTI.ipynb
07.08.2022  14:57        15 673 470 r_wallstreetbets_new_2022.csv
07.08.2022  18:19            17 121 Topic modeling.ipynb
               3 файлов     15 702 739 байт
               3 папок  50 973 077 504 байт свободно


In [47]:
df = pd.read_csv(os.path.join("../data/r_wallstreetbets_new_2022.csv"))

In [48]:
df.columns

Index(['Unnamed: 0', 'author', 'created_utc', 'domain', 'downs', 'id', 'kind',
       'link_flair_css_class', 'num_comments', 'score', 'selftext',
       'subreddit', 'title', 'ups', 'upvote_ratio'],
      dtype='object')

In [ ]:
config_path = os.path.join('../config/params_all.yaml')
config = yaml.safe_load(open(config_path))['train']


In [ ]:
!dir ..\data

In [ ]:
import os
import logging
import yaml
from pyyoutube import Api
import json
import requests
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
import re
import numpy as np
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import SpectralClustering
from sklearn.metrics import f1_score, silhouette_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.tracking import MlflowClient

def get_data(YOUTUBE_API_KEY, videoId, maxResults, nextPageToken):
    """
    Получение информации со страницы с видео
    """
    YOUTUBE_URI = 'https://www.googleapis.com/youtube/v3/commentThreads?key={KEY}&textFormat=plainText&' + \
        'part=snippet&videoId={videoId}&maxResults={maxResults}&pageToken={nextPageToken}'
    format_youtube_uri = YOUTUBE_URI.format(KEY=YOUTUBE_API_KEY,
                                            videoId=videoId,
                                            maxResults=maxResults,
                                            nextPageToken=nextPageToken)
    content = requests.get(format_youtube_uri).text
    data = json.loads(content)
    return data


def get_text_of_comment(data):
    """
    Получение комментариев из полученных данных под одним видео
    """
    comms = set()
    for item in data['items']:
        comm = item['snippet']['topLevelComment']['snippet']['textDisplay']
        comms.add(comm)
    return comms


def get_all_comments(YOUTUBE_API_KEY, query, count_video=10, limit=30, maxResults=10, nextPageToken=''):
    """
    Выгрузка maxResults комментариев
    """
    api = Api(api_key=YOUTUBE_API_KEY)
    video_by_keywords = api.search_by_keywords(q=query,
                                               search_type=["video"],
                                               count=count_video,
                                               limit=limit)
    videoId = [x.id.videoId for x in video_by_keywords.items]

    comments_all = []
    for id_video in videoId:
        try:
            data = get_data(YOUTUBE_API_KEY,
                            id_video,
                            maxResults=maxResults,
                            nextPageToken=nextPageToken)
            comment = list(get_text_of_comment(data))
            comments_all.append(comment)
        except:
            continue
    comments = sum(comments_all, [])
    return comments

!ls ../config

config_path = os.path.join('../config/params_all.yaml')
config = yaml.safe_load(open(config_path))['train']

SEED = config['SEED']

config

comments = get_all_comments(**config['comments'])

comments[:10]

def remove_emoji(string):
    """
    Удаление эмоджи из текста
    """
    emoji_pattern = re.compile("["u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def remove_links(string):
    """
    Удаление ссылок
    """
    string = re.sub(r'http\S+', '', string)  # remove http links
    string = re.sub(r'bit.ly/\S+', '', string)  # rempve bitly links
    string = re.sub(r'www\S+', '', string)  # rempve bitly links
    string = string.strip('[link]')  # remove [links]
    return string


def preprocessing(string, stopwords, stem):
    """
    Простой препроцессинг текста, очистка, лематизация, удаление коротких слов
    """
    string = remove_emoji(string)
    string = remove_links(string)

    # удаление символов "\r\n"
    str_pattern = re.compile("\r\n")
    string = str_pattern.sub(r'', string)

    # очистка текста от символов
    string = re.sub('(((?![а-яА-Я ]).)+)', ' ', string)
    # лематизация
    string = ' '.join([
        re.sub('\\n', '', ' '.join(stem.lemmatize(s))).strip()
        for s in string.split()
    ])
    # удаляем слова короче 3 символов
    string = ' '.join([s for s in string.split() if len(s) > 3])
    # удаляем стоп-слова
    string = ' '.join([s for s in string.split() if s not in stopwords])
    return string


def get_clean_text(data, stopwords):
    """
    Получение текста в преобразованной после очистки
    матричном виде, а также модель векторизации
    """
    # Простой препроцессинг текста
    stem = Mystem()
    comments = [preprocessing(x, stopwords, stem) for x in data]
    # Удаление комментов, которые имеют меньше, чем 5 слов
    comments = [y for y in comments if len(y.split()) > 5]
    #common_texts = [i.split(' ') for i in comments]
    return comments


def vectorize_text(data, tfidf):
    """
    Получение матрицы кол-ва слов в комменариях
    Очистка от пустых строк
    """
    # Векторизация
    X_matrix = tfidf.transform(data).toarray()
    # Удаляем строки в матрице с пустыми значениями
    mask = (np.nan_to_num(X_matrix) != 0).any(axis=1)
    return X_matrix[mask]

comments_clean = get_clean_text(comments, stopwords.words(config['stopwords']))
tfidf = TfidfVectorizer(**config['tf_model']).fit(comments_clean)

config

comments_clean[:10]

X_matrix = vectorize_text(comments_clean, tfidf)

X_matrix.shape

tfidf.get_feature_names()[:10]

def get_clusters(data, count_max_clusters, random_state, affinity,
                 silhouette_metric):
    """
    Подбор наилучшего числа кластеров, возвращает полученные кластера тематик
    """
    cluster_labels = {}
    silhouette_mean = []

    for i in range(2, count_max_clusters, 1):
        clf = SpectralClustering(n_clusters=i,
                                 affinity=affinity,
                                 random_state=random_state)
        #clf = KMeans(n_clusters=n, max_iter=1000, n_init=1)
        clf.fit(data)
        labels = clf.labels_
        cluster_labels[i] = labels
        silhouette_mean.append(
            silhouette_score(data, labels, metric=silhouette_metric))
    n_clusters = silhouette_mean.index(max(silhouette_mean)) + 2
    return cluster_labels[n_clusters]


def get_f1_score(y_test, y_pred, unique_cluster_labels):
    """
    Возращает результат обучения классификатора по тематикам
    """
    return f1_score(
        y_test, y_pred,
        average='macro') \
        if len(unique_cluster_labels) > 2 \
        else f1_score(y_test, y_pred)

cluster_labels = get_clusters(X_matrix,
                                 random_state=SEED,
                                 **config['clustering'])

config

cluster_labels[:10]

X_train, X_test, y_train, y_test = train_test_split(X_matrix,
                                                    cluster_labels,
                                                    **config['cross_val'],
                                                    random_state=SEED)

clf_lr = LogisticRegression(**config['model'])

%%bash
export MLFLOW_REGISTRY_URI=../mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(config['name_experiment'])
with mlflow.start_run():
    clf_lr.fit(X_train, y_train)
    print(clf_lr.predict_proba(X_test))

    # Логирование модели и параметров
    mlflow.log_param(
        'f1', get_f1_score(y_test, clf_lr.predict(X_test),
                           set(cluster_labels)))
    mlflow.sklearn.log_model(
        tfidf,
        artifact_path="vector",
        registered_model_name=f"{config['model_vec']}")
    mlflow.sklearn.log_model(
        clf_lr,
        artifact_path='model_lr',
        registered_model_name=f"{config['model_lr']}")
    mlflow.end_run()

mlflow.get_artifact_uri()

def get_version_model(config_name, client):
    """
    Получение последней версии модели из MLFlow
    """
    dict_push = {}
    for count, value in enumerate(
        client.search_model_versions(f"name='{config_name}'")):
        # client.list_registered_models()):
        # Все версии модели
        dict_push[count] = value
    return dict(list(dict_push.items())[-1][1])['version']

client = MlflowClient()
last_version_lr = get_version_model(config['model_lr'], client)
last_version_vec = get_version_model(config['model_vec'], client)

last_version_lr

last_version_vec

